<a href="https://colab.research.google.com/github/zz2585/CommonOwnerReplication/blob/master/Amihud_2002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The paper examines return-illiquidity relationship over time. 
# Over time, ex ante stock excess return is increasing in the expected illiquidity of the stock market 
# Illiquidity ratio: ILLIQ defined as the daily ratio of absolute stock return to dollar volume, average over xx days 
# The ratio gives the daily price impact of the order flow 
# consider verify the findings using spreads and PIN 

\begin{align}
ILLILQ_{iy} = \frac{1}{D_{iy}}*\sum \limits_{t=1}^{D_{iy}} \frac{\mid{R_{iyd}}\mid}{VOLD_{ivyd}}
\end{align}

where $D_{iy}$ is the number of trading days for stock $i$ in year $y$.

In [ ]:
import pandas as pd
import numpy as np
import wrds
from appelpy.linear_model import OLS


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install appelpy 

In [ ]:
# sample period: 1963-1997 
# sample: NYSE-traded stocks (i.e., EXCHCD==1 or 31)

#connect to wrds 
conn=wrds.Connection()



Enter your WRDS username [root]:zz2585
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [ ]:
# get crsp monthly data
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc, a.vol
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1963' and '12/31/1997'
                      and b.exchcd =1 or b.exchcd =31
                      """, date_cols=['date']) 


# get delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt, dlstcd
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])


In [ ]:
crsp.describe()

,permno,permco,shrcd,exchcd,ret,retx,shrout,prc,vol,dlret,dlstcd,retadj,ILLIQ,yr,key
count,660990.000000,660990.000000,660990.000000,660990.000000,660990.000000,658877.000000,6.609900e+05,660990.000000,6.609900e+05,319.000000,325.000000,660990.000000,6.609900e+05,660990.000000,660990.000000
mean,41194.329541,20313.578299,12.720362,1.001589,0.012528,0.009616,3.208344e+04,29.933656,1.582012e+04,0.015079,265.818462,0.012518,9.004769e-06,1981.944123,43176.273664
std,21308.362148,6630.289444,8.627656,0.218296,0.110178,0.110489,7.991515e+04,281.348613,4.555754e+04,0.162957,99.842906,0.110184,4.505744e-04,10.242245,21313.968353
min,10006.000000,4.000000,10.000000,1.000000,-0.937500,-0.937500,0.000000e+00,0.015625,1.000000e+00,-1.000000,200.000000,-1.000000,0.000000e+00,1926.000000,11969.000000
25%,22787.000000,20549.000000,10.000000,1.000000,-0.042893,-0.046414,4.702000e+03,13.125000,9.310000e+02,-0.000262,231.000000,-0.042869,1.160560e-07,1973.000000,24763.000000
50%,39925.000000,21430.000000,11.000000,1.000000,0.006329,0.000000,1.122400e+04,22.250000,3.121000e+03,0.008065,231.000000,0.006329,6.647486e-07,1983.000000,41905.000000
75%,57665.000000,23126.000000,11.000000,1.000000,0.060606,0.057935,2.914000e+04,34.625000,1.161100e+04,0.028826,241.000000,0.060606,3.161549e-06,1991.000000,59658.750000
max,93201.000000,56285.000000,71.000000,31.000000,11.000000,11.000000,3.272726e+06,47200.000000,2.641605e+06,1.250000,588.000000,11.000000,2.666667e-01,1999.000000,95191.000000


In [ ]:
# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['jdate']=crsp_m['date']+pd.offsets.MonthEnd(0)

dlret['dlstcd']=dlret['dlstcd'].astype(int)
dlret['permno']=dlret['permno'].astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+pd.offsets.MonthEnd(0) 

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])

crsp['ret']=crsp['ret'].fillna(0)


In [ ]:
# adjust for delisting returns following Shumway(1997)
crsp['retadj'] = np.where(crsp.dlstcd.isna(), crsp.ret, np.nan)
crsp['retadj'] = np.where((crsp.dlstcd.notna())&(crsp.dlret.notna()), crsp.dlret, crsp['retadj'])
crsp['retadj'] = np.where((crsp.dlstcd==500) | (crsp.dlstcd==520)| (crsp.dlstcd==580)| (crsp.dlstcd==584)|((crsp.dlstcd >= 551) & (crsp.dlstcd <= 574)), -0.3, crsp['retadj'])
crsp['retadj'] = np.where(crsp['retadj'].isna(), -1, crsp['retadj'])


In [ ]:
#to be deleted 
# retadj factors in the delisting returns
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

In [ ]:
# filter data based on several criteria

# drop ADRs
crsp = crsp[~((crsp.shrcd==30) | (crsp.shrcd==31))]

# drop observations where vol is null
crsp = crsp[(crsp['vol'] > 0)& (crsp['prc'] > 0)]

# calculate Amihud ILLIQ measure
crsp['ILLIQ'] = abs(crsp['ret']) / (crsp['prc'] * crsp['vol'])

# (i) The stock has return and volume data for more than 200 days during year y. Also, the stock must be listed at the end of year y. 
# zip permno and delist year in both dataset to remove delist firms 
dlret['key'] = dlret['permno'] + dlret['dlstdt'].dt.year
crsp['key'] = crsp['permno'] + crsp['date'].dt.year
crsp_1 = crsp[~crsp.key.isin(dlret.key)].reset_index().drop(['jdate','dlret','dlstdt','dlstcd','index', 'key'],axis=1)

# count CRSP daily observations
crsp_d = conn.raw_sql(f"""
                        select permno, date, prc, abs(prc*shrout) as mcap, ret from crsp.dsf
                        where date between '01/01/1963' and '12/31/1997'
                        """, date_cols = ['date'])


crsp_1['yr'] = crsp_1['date'].dt.year
obs = crsp_1.groupby(['yr', 'permno']).date.count().reset_index()


# (ii) The stock price is greater than $5 at the end of year y.
# (iii) The stock has data on market capitalization at the end of year y 􏰣 1 in the CRSP database. 
# (iv) Stock-year ILLIQ is winsorized at the 1% level 


In [ ]:
df_trading_days_200 = df_counts[df_counts['counts'] > 200][['year', 'PERMNO']].reset_index(drop=True)
df_div_amihud = df_notnull.groupby(['year', 'PERMNO']).sum().reset_index()[['year', 'PERMNO', 'amihud_d', 'DIVAMT']]
df_div_amihud['counts'] = df_notnull.groupby(['year', 'PERMNO']).size().reset_index(name='counts')['counts']
df_div_amihud['sdret'] = df_notnull.groupby(['year', 'PERMNO']).std().reset_index()['RET'] * 100
df_step1 = df_trading_days_200.merge(df_div_amihud, 'left', on=['year', 'PERMNO'])